# Part 1

In [1]:
import pandas as pd
import requests
import time
import json
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime

In [2]:
# Путь к файлу
filepath  = Path('/content/drive/MyDrive/df_pred.csv')

In [3]:
# Загрузка файла
df_pred = pd.read_csv(filepath)

In [4]:
# Загрузка файла
df_pred_num = pd.read_csv(filepath)

In [5]:
df_pred.head(5)

,match_id,radiant_win,duration,radiant_team,dire_team
0,7124356205,False,607,"66,19,76,128,13","11,4,14,100,108"
1,7124377210,False,905,"73,106,114,5,40","46,93,101,2,62"
2,7124372112,True,1291,"93,27,123,14,19","25,43,17,72,12"
3,7124366900,True,1321,"107,70,30,126,36","75,71,2,11,114"
4,7124364915,True,1000,"48,110,104,90,11","135,93,36,37,43"


In [6]:
h = requests.get(f'https://api.opendota.com/api/heroes')
heroes = h.json()
df_heroes = pd.DataFrame.from_dict(heroes)
df_heroes

,id,name,localized_name,primary_attr,attack_type,roles,legs
0,1,npc_dota_hero_antimage,Anti-Mage,agi,Melee,"[Carry, Escape, Nuker]",2
1,2,npc_dota_hero_axe,Axe,str,Melee,"[Initiator, Durable, Disabler, Carry]",2
2,3,npc_dota_hero_bane,Bane,all,Ranged,"[Support, Disabler, Nuker, Durable]",4
3,4,npc_dota_hero_bloodseeker,Bloodseeker,agi,Melee,"[Carry, Disabler, Nuker, Initiator]",2
4,5,npc_dota_hero_crystal_maiden,Crystal Maiden,int,Ranged,"[Support, Disabler, Nuker]",2
...,...,...,...,...,...,...,...
119,129,npc_dota_hero_mars,Mars,str,Melee,"[Carry, Initiator, Disabler, Durable]",2
120,135,npc_dota_hero_dawnbreaker,Dawnbreaker,str,Melee,"[Carry, Durable]",2
121,136,npc_dota_hero_marci,Marci,all,Melee,"[Support, Carry, Initiator, Disabler, Escape]",2
122,137,npc_dota_hero_primal_beast,Primal Beast,str,Melee,"[Initiator, Durable, Disabler]",2


In [7]:
df_hero_id_name = df_heroes[['id', 'localized_name']]
df_hero_id_name = df_hero_id_name.rename({'id': 'id_hero'}, axis=1)
df_hero = df_hero_id_name

In [8]:
def replace_hero_names(df_pred, df_hero):
    hero_names = df_hero.set_index('id_hero')['localized_name'].to_dict() # Создаем словарь для быстрого доступа
   
    radiant_team_col = 'radiant_team'
    for idx, row in df_pred.iterrows():
        radiant_team = row.get(radiant_team_col, '') # Получаем значение нужной колонки строки
        
        # Заменяем id на имя героя, используя словарь
        radiant_team_names = [hero_names.get(int(hero_id), '') for hero_id in radiant_team.split(',') if hero_id]
        
        # Обновляем значение в таблице
        df_pred.at[idx, radiant_team_col] = ','.join(radiant_team_names)
        radiant_team_col = 'radiant_team'
    
    dire_team_col = 'dire_team'
    for idx, row in df_pred.iterrows():
        dire_team = row.get(dire_team_col, '') # Получаем значение нужной колонки строки
        
        # Заменяем id на имя героя, используя словарь
        dire_team_names = [hero_names.get(int(hero_id), '') for hero_id in dire_team.split(',') if hero_id]
        
        # Обновляем значение в таблице
        df_pred.at[idx, dire_team_col] = ','.join(dire_team_names)
        
    return df_pred

In [9]:
df_pred.head(1)

,match_id,radiant_win,duration,radiant_team,dire_team
0,7124356205,False,607,"66,19,76,128,13","11,4,14,100,108"


In [10]:
df_hero.head(1)

,id_hero,localized_name
0,1,Anti-Mage


In [11]:
df_replaced = replace_hero_names(df_pred, df_hero)
print(df_replaced)

        match_id  radiant_win  duration  \
0     7124356205        False       607   
1     7124377210        False       905   
2     7124372112         True      1291   
3     7124366900         True      1321   
4     7124364915         True      1000   
...          ...          ...       ...   
3452  7153210744        False      2596   
3453  7153379459         True      2966   
3454  7153302648        False      2442   
3455  7153446429        False      2233   
3456  7153241167         True      2564   

                                           radiant_team  \
0            Chen,Tiny,Outworld Destroyer,Snapfire,Puck   
1     Alchemist,Ember Spirit,Monkey King,Crystal Mai...   
2               Slark,Shadow Shaman,Hoodwink,Pudge,Tiny   
3     Earth Spirit,Ursa,Witch Doctor,Void Spirit,Nec...   
4     Luna,Phoenix,Legion Commander,Keeper of the Li...   
...                                                 ...   
3452  Enchantress,Bounty Hunter,Rubick,Slark,Legion ...   
3453  Batri

In [12]:
df_pred.head(5)

,match_id,radiant_win,duration,radiant_team,dire_team
0,7124356205,False,607,"Chen,Tiny,Outworld Destroyer,Snapfire,Puck","Shadow Fiend,Bloodseeker,Pudge,Tusk,Underlord"
1,7124377210,False,905,"Alchemist,Ember Spirit,Monkey King,Crystal Mai...","Templar Assassin,Slark,Skywrath Mage,Axe,Bount..."
2,7124372112,True,1291,"Slark,Shadow Shaman,Hoodwink,Pudge,Tiny","Lina,Death Prophet,Storm Spirit,Gyrocopter,Pha..."
3,7124366900,True,1321,"Earth Spirit,Ursa,Witch Doctor,Void Spirit,Nec...","Silencer,Spirit Breaker,Axe,Shadow Fiend,Monke..."
4,7124364915,True,1000,"Luna,Phoenix,Legion Commander,Keeper of the Li...","Dawnbreaker,Slark,Necrophos,Warlock,Death Prophet"


In [13]:
# Сплит колонок
df_pred[['dtp_1', 'dtp_2', 'dtp_3', 'dtp_4', 'dtp_5']] = df_pred['dire_team']. str.split(',' , expand= True )
df_pred[['rtp_1', 'rtp_2', 'rtp_3', 'rtp_4', 'rtp_5']] = df_pred['radiant_team']. str.split(',' , expand= True )

In [14]:
df_pred.radiant_win = df_pred.radiant_win.replace({ True : 1 , False : 0 })
#df_pred_num.radiant_win = df_pred_num.radiant_win.replace({ True : 1 , False : 0 })

# Catboost

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
import seaborn as sns

In [16]:
train, test = train_test_split(df_pred, train_size=0.6,random_state=42)

In [17]:
val,test = train_test_split(test, train_size=0.5,random_state=42)

In [18]:
! pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 8.1 MB/s eta 0:00:00


In [19]:
from catboost import CatBoostRegressor

In [20]:
train.columns

Index(['match_id', 'radiant_win', 'duration', 'radiant_team', 'dire_team',
       'dtp_1', 'dtp_2', 'dtp_3', 'dtp_4', 'dtp_5', 'rtp_1', 'rtp_2', 'rtp_3',
       'rtp_4', 'rtp_5'],
      dtype='object')

In [21]:
x = ['duration', 'dtp_1', 'dtp_2', 'dtp_3', 'dtp_4', 'dtp_5', 'rtp_1', 'rtp_2', 'rtp_3',
       'rtp_4', 'rtp_5']
cat_features=['dtp_1', 'dtp_2', 'dtp_3', 'dtp_4', 'dtp_5', 'rtp_1', 'rtp_2', 'rtp_3',
       'rtp_4', 'rtp_5']
y = ['radiant_win']

In [22]:
parametrs = {'cat_features' : cat_features, 
             'eval_metric':'MAPE',
             'learning_rate' : 0.01,
             'random_seed':42,
             'verbose':100}

In [23]:
model = CatBoostRegressor(**parametrs)

In [24]:
model.fit(train[x],train[y], eval_set=(val[x],val[y]))

0:	learn: 0.4974471	test: 0.4991421	best: 0.4991421 (0)	total: 59.9ms	remaining: 59.8s
100:	learn: 0.4891114	test: 0.4994853	best: 0.4990075 (22)	total: 676ms	remaining: 6.02s
200:	learn: 0.4811810	test: 0.4994271	best: 0.4990075 (22)	total: 1.32s	remaining: 5.26s
300:	learn: 0.4737725	test: 0.4995755	best: 0.4990075 (22)	total: 1.99s	remaining: 4.62s
400:	learn: 0.4669010	test: 0.4991810	best: 0.4990075 (22)	total: 2.9s	remaining: 4.34s
500:	learn: 0.4609706	test: 0.4988676	best: 0.4988676 (500)	total: 3.69s	remaining: 3.68s
600:	learn: 0.4546419	test: 0.4985923	best: 0.4985923 (600)	total: 4.51s	remaining: 2.99s
700:	learn: 0.4481396	test: 0.4986023	best: 0.4985238 (673)	total: 5.41s	remaining: 2.31s
800:	learn: 0.4419521	test: 0.4986517	best: 0.4985238 (673)	total: 6.46s	remaining: 1.6s
900:	learn: 0.4356079	test: 0.4989349	best: 0.4985238 (673)	total: 8.15s	remaining: 895ms
999:	learn: 0.4292696	test: 0.4989769	best: 0.4985238 (673)	total: 9.8s	remaining: 0us

bestTest = 0.49852378

In [25]:
test['win_pred'] = model.predict(test[x])

In [26]:
def error(y_true, y_pred):
  print(mean_absolute_error(y_true, y_pred))
  print(mean_absolute_percentage_error(y_true, y_pred))

In [27]:
error(test['radiant_win'], test['win_pred'])

0.4969690688820247
1159504798914984.8


In [28]:
test[['radiant_win', 'win_pred']]

,radiant_win,win_pred
276,0,0.557245
1067,1,0.555376
1407,0,0.459476
3052,1,0.575777
2865,1,0.560421
...,...,...
233,0,0.574722
3279,1,0.561868
2431,1,0.541244
1192,0,0.628111


In [29]:
test.radiant_win.mean()

0.5173410404624278

In [30]:
test.win_pred.mean()

0.5352955415479144

In [31]:
test.sort_values(by='win_pred')

,match_id,radiant_win,duration,radiant_team,dire_team,dtp_1,dtp_2,dtp_3,dtp_4,dtp_5,rtp_1,rtp_2,rtp_3,rtp_4,rtp_5,win_pred
2223,7160876906,0,2574,"Crystal Maiden,Nyx Assassin,Terrorblade,Doom,V...","Rubick,Monkey King,Spirit Breaker,Juggernaut,P...",Rubick,Monkey King,Spirit Breaker,Juggernaut,Pangolier,Crystal Maiden,Nyx Assassin,Terrorblade,Doom,Void Spirit,0.396315
1898,7156865726,1,2282,"Medusa,Hoodwink,Dawnbreaker,Wraith King,Queen ...","Rubick,Oracle,Bloodseeker,Windranger,Outworld ...",Rubick,Oracle,Bloodseeker,Windranger,Outworld Destroyer,Medusa,Hoodwink,Dawnbreaker,Wraith King,Queen of Pain,0.401477
1468,7165057240,0,2817,"Skywrath Mage,Ember Spirit,Grimstroke,Juggerna...","Doom,Pugna,Dark Willow,Void Spirit,Medusa",Doom,Pugna,Dark Willow,Void Spirit,Medusa,Skywrath Mage,Ember Spirit,Grimstroke,Juggernaut,Batrider,0.415654
1378,7130634268,1,2623,"Crystal Maiden,Terrorblade,Clockwerk,Dark Seer...","Rubick,Brewmaster,Disruptor,Faceless Void,Alch...",Rubick,Brewmaster,Disruptor,Faceless Void,Alchemist,Crystal Maiden,Terrorblade,Clockwerk,Dark Seer,Void Spirit,0.420216
2129,7170108895,0,1676,"Crystal Maiden,Dark Willow,Dragon Knight,Templ...","Bounty Hunter,Rubick,Centaur Warrunner,Bloodse...",Bounty Hunter,Rubick,Centaur Warrunner,Bloodseeker,Storm Spirit,Crystal Maiden,Dark Willow,Dragon Knight,Templar Assassin,Earthshaker,0.421174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1234,7137654773,1,1347,"Medusa,Bounty Hunter,Venomancer,Naga Siren,Rubick","Skywrath Mage,Nyx Assassin,Batrider,Slark,Sand...",Skywrath Mage,Nyx Assassin,Batrider,Slark,Sand King,Medusa,Bounty Hunter,Venomancer,Naga Siren,Rubick,0.631488
239,7139919413,0,925,"Underlord,Keeper of the Light,Windranger,Warlo...","Phantom Lancer,Doom,Undying,Enchantress,Void S...",Phantom Lancer,Doom,Undying,Enchantress,Void Spirit,Underlord,Keeper of the Light,Windranger,Warlock,Riki,0.635751
2986,7148539576,0,2203,"Bounty Hunter,Legion Commander,Venomancer,Disr...","Skywrath Mage,Dawnbreaker,Undying,Juggernaut,Axe",Skywrath Mage,Dawnbreaker,Undying,Juggernaut,Axe,Bounty Hunter,Legion Commander,Venomancer,Disruptor,Anti-Mage,0.636963
3255,7175264073,0,2217,"Techies,Legion Commander,Void Spirit,Mirana,Riki","Magnus,Batrider,Oracle,Winter Wyvern,Chaos Knight",Magnus,Batrider,Oracle,Winter Wyvern,Chaos Knight,Techies,Legion Commander,Void Spirit,Mirana,Riki,0.652657


# CatBoostClassifier

In [32]:
from catboost import CatBoostClassifier

In [33]:
x = ['duration', 'dtp_1', 'dtp_2', 'dtp_3', 'dtp_4', 'dtp_5', 'rtp_1', 'rtp_2', 'rtp_3',
       'rtp_4', 'rtp_5']
cat_features=['dtp_1', 'dtp_2', 'dtp_3', 'dtp_4', 'dtp_5', 'rtp_1', 'rtp_2', 'rtp_3',
       'rtp_4', 'rtp_5']
y = ['radiant_win']

In [34]:
from catboost import Pool

train_data = Pool(data=train[x],
                  label=train[y],
                  cat_features=cat_features
                 )

valid_data = Pool(data=val[x],
                  label=val[y],
                  cat_features=cat_features
                 )

In [35]:
parametrs = {'random_seed':42,
             'learning_rate' : 0.001,
             'verbose':100}

In [36]:
model = CatBoostClassifier(**parametrs)

In [37]:
model.fit(train_data, eval_set=valid_data)

0:	learn: 0.6931167	test: 0.6931465	best: 0.6931465 (0)	total: 39.2ms	remaining: 39.1s
100:	learn: 0.6904685	test: 0.6929919	best: 0.6929820 (96)	total: 1.21s	remaining: 10.7s
200:	learn: 0.6879593	test: 0.6928475	best: 0.6928362 (191)	total: 2.15s	remaining: 8.55s
300:	learn: 0.6854675	test: 0.6927471	best: 0.6927467 (297)	total: 3.01s	remaining: 7s
400:	learn: 0.6829426	test: 0.6927245	best: 0.6927158 (391)	total: 3.86s	remaining: 5.76s
500:	learn: 0.6807266	test: 0.6927465	best: 0.6926939 (426)	total: 4.77s	remaining: 4.75s
600:	learn: 0.6783085	test: 0.6927718	best: 0.6926939 (426)	total: 5.66s	remaining: 3.76s
700:	learn: 0.6760480	test: 0.6929271	best: 0.6926939 (426)	total: 6.52s	remaining: 2.78s
800:	learn: 0.6738961	test: 0.6930056	best: 0.6926939 (426)	total: 7.42s	remaining: 1.84s
900:	learn: 0.6716914	test: 0.6931056	best: 0.6926939 (426)	total: 8.26s	remaining: 907ms
999:	learn: 0.6696009	test: 0.6931930	best: 0.6926939 (426)	total: 10.1s	remaining: 0us

bestTest = 0.69269

In [38]:
test['win_pred_score'] = model.predict_proba(test[x])[:,1]

In [39]:
test['win_score_round'] = round(test['win_pred_score'])
test['win_pred_round'] = round(test['win_pred'])

In [40]:
test.sort_values(by = 'win_pred', ascending=False)

,match_id,radiant_win,duration,radiant_team,dire_team,dtp_1,dtp_2,dtp_3,dtp_4,dtp_5,rtp_1,rtp_2,rtp_3,rtp_4,rtp_5,win_pred,win_pred_score,win_score_round,win_pred_round
3337,7160918227,0,1315,"Omniknight,Clinkz,Spirit Breaker,Disruptor,Riki","Undying,Beastmaster,Skywrath Mage,Faceless Voi...",Undying,Beastmaster,Skywrath Mage,Faceless Void,Huskar,Omniknight,Clinkz,Spirit Breaker,Disruptor,Riki,0.662164,0.526283,1.0,1.0
3255,7175264073,0,2217,"Techies,Legion Commander,Void Spirit,Mirana,Riki","Magnus,Batrider,Oracle,Winter Wyvern,Chaos Knight",Magnus,Batrider,Oracle,Winter Wyvern,Chaos Knight,Techies,Legion Commander,Void Spirit,Mirana,Riki,0.652657,0.522277,1.0,1.0
2986,7148539576,0,2203,"Bounty Hunter,Legion Commander,Venomancer,Disr...","Skywrath Mage,Dawnbreaker,Undying,Juggernaut,Axe",Skywrath Mage,Dawnbreaker,Undying,Juggernaut,Axe,Bounty Hunter,Legion Commander,Venomancer,Disruptor,Anti-Mage,0.636963,0.521622,1.0,1.0
239,7139919413,0,925,"Underlord,Keeper of the Light,Windranger,Warlo...","Phantom Lancer,Doom,Undying,Enchantress,Void S...",Phantom Lancer,Doom,Undying,Enchantress,Void Spirit,Underlord,Keeper of the Light,Windranger,Warlock,Riki,0.635751,0.521801,1.0,1.0
1234,7137654773,1,1347,"Medusa,Bounty Hunter,Venomancer,Naga Siren,Rubick","Skywrath Mage,Nyx Assassin,Batrider,Slark,Sand...",Skywrath Mage,Nyx Assassin,Batrider,Slark,Sand King,Medusa,Bounty Hunter,Venomancer,Naga Siren,Rubick,0.631488,0.523107,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2129,7170108895,0,1676,"Crystal Maiden,Dark Willow,Dragon Knight,Templ...","Bounty Hunter,Rubick,Centaur Warrunner,Bloodse...",Bounty Hunter,Rubick,Centaur Warrunner,Bloodseeker,Storm Spirit,Crystal Maiden,Dark Willow,Dragon Knight,Templar Assassin,Earthshaker,0.421174,0.498556,0.0,0.0
1378,7130634268,1,2623,"Crystal Maiden,Terrorblade,Clockwerk,Dark Seer...","Rubick,Brewmaster,Disruptor,Faceless Void,Alch...",Rubick,Brewmaster,Disruptor,Faceless Void,Alchemist,Crystal Maiden,Terrorblade,Clockwerk,Dark Seer,Void Spirit,0.420216,0.498581,0.0,0.0
1468,7165057240,0,2817,"Skywrath Mage,Ember Spirit,Grimstroke,Juggerna...","Doom,Pugna,Dark Willow,Void Spirit,Medusa",Doom,Pugna,Dark Willow,Void Spirit,Medusa,Skywrath Mage,Ember Spirit,Grimstroke,Juggernaut,Batrider,0.415654,0.499927,0.0,0.0
1898,7156865726,1,2282,"Medusa,Hoodwink,Dawnbreaker,Wraith King,Queen ...","Rubick,Oracle,Bloodseeker,Windranger,Outworld ...",Rubick,Oracle,Bloodseeker,Windranger,Outworld Destroyer,Medusa,Hoodwink,Dawnbreaker,Wraith King,Queen of Pain,0.401477,0.496474,0.0,0.0


In [41]:
error(test['radiant_win'], test['win_pred_score'])

0.49895953316058494
1113018289656641.2


In [42]:
error(test['radiant_win'], test['win_pred_round'])

0.4725433526011561
1685595814290402.5


In [43]:
error(test['radiant_win'], test['win_score_round'])

0.4725433526011561
2108621790849769.8


In [44]:
error(test['win_pred_round'], test['win_score_round'])

0.18786127167630057
846051953118734.8


In [45]:
test[['radiant_win', 'win_pred_round', 'win_score_round']]

,radiant_win,win_pred_round,win_score_round
276,0,1.0,1.0
1067,1,1.0,1.0
1407,0,0.0,1.0
3052,1,1.0,1.0
2865,1,1.0,1.0
...,...,...,...
233,0,1.0,1.0
3279,1,1.0,1.0
2431,1,1.0,1.0
1192,0,1.0,1.0


In [46]:
df_pred.corr()['radiant_win'].sort_values()

<ipython-input-46-4d7727073f85>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_pred.corr()['radiant_win'].sort_values()


duration      -0.035462
match_id       0.008329
radiant_win    1.000000
Name: radiant_win, dtype: float64

In [47]:
df_pred

,match_id,radiant_win,duration,radiant_team,dire_team,dtp_1,dtp_2,dtp_3,dtp_4,dtp_5,rtp_1,rtp_2,rtp_3,rtp_4,rtp_5
0,7124356205,0,607,"Chen,Tiny,Outworld Destroyer,Snapfire,Puck","Shadow Fiend,Bloodseeker,Pudge,Tusk,Underlord",Shadow Fiend,Bloodseeker,Pudge,Tusk,Underlord,Chen,Tiny,Outworld Destroyer,Snapfire,Puck
1,7124377210,0,905,"Alchemist,Ember Spirit,Monkey King,Crystal Mai...","Templar Assassin,Slark,Skywrath Mage,Axe,Bount...",Templar Assassin,Slark,Skywrath Mage,Axe,Bounty Hunter,Alchemist,Ember Spirit,Monkey King,Crystal Maiden,Venomancer
2,7124372112,1,1291,"Slark,Shadow Shaman,Hoodwink,Pudge,Tiny","Lina,Death Prophet,Storm Spirit,Gyrocopter,Pha...",Lina,Death Prophet,Storm Spirit,Gyrocopter,Phantom Lancer,Slark,Shadow Shaman,Hoodwink,Pudge,Tiny
3,7124366900,1,1321,"Earth Spirit,Ursa,Witch Doctor,Void Spirit,Nec...","Silencer,Spirit Breaker,Axe,Shadow Fiend,Monke...",Silencer,Spirit Breaker,Axe,Shadow Fiend,Monkey King,Earth Spirit,Ursa,Witch Doctor,Void Spirit,Necrophos
4,7124364915,1,1000,"Luna,Phoenix,Legion Commander,Keeper of the Li...","Dawnbreaker,Slark,Necrophos,Warlock,Death Prophet",Dawnbreaker,Slark,Necrophos,Warlock,Death Prophet,Luna,Phoenix,Legion Commander,Keeper of the Light,Shadow Fiend
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3452,7153210744,0,2596,"Enchantress,Bounty Hunter,Rubick,Slark,Legion ...","Keeper of the Light,Underlord,Crystal Maiden,T...",Keeper of the Light,Underlord,Crystal Maiden,Terrorblade,Batrider,Enchantress,Bounty Hunter,Rubick,Slark,Legion Commander
3453,7153379459,1,2966,"Batrider,Kunkka,Witch Doctor,Nyx Assassin,Phan...","Oracle,Queen of Pain,Rubick,Slark,Axe",Oracle,Queen of Pain,Rubick,Slark,Axe,Batrider,Kunkka,Witch Doctor,Nyx Assassin,Phantom Assassin
3454,7153302648,0,2442,"Keeper of the Light,Ursa,Crystal Maiden,Tiny,B...","Doom,Skywrath Mage,Rubick,Templar Assassin,Leg...",Doom,Skywrath Mage,Rubick,Templar Assassin,Legion Commander,Keeper of the Light,Ursa,Crystal Maiden,Tiny,Beastmaster
3455,7153446429,0,2233,"Underlord,Rubick,Lich,Bloodseeker,Kunkka","Keeper of the Light,Monkey King,Dark Seer,Cloc...",Keeper of the Light,Monkey King,Dark Seer,Clockwerk,Batrider,Underlord,Rubick,Lich,Bloodseeker,Kunkka
